# Aula 5 - Espaços de features polinomiais - aumentando a complexidade de hipóteses

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Espaços de features polinomiais

In [ ]:
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
from ml_utils import *

In [ ]:
dir()

____
____
____

_____

## 1) Espaços de features polinomiais

Muitas vezes, temos dados que simplesmente não se ajustam às hipóteses simples, lineares, que conhecemos até o momento.

Quando isso acontece, é muito provável que soframos **underfitting**, pois uma forma funcional demasiadamente simples de uma hipótese pode não ser capaz de capturar o comportamento de uma função teórica $\mathcal{F}$ mais complexa, conforme refletido pela amostra.

Nestes casos, a solução é simples: basta escolhermos hipóteses mais complexas!

Pra começar nosso estudo, vamos utilizar dados artificiais bem simples: 


In [ ]:
np.random.seed(42)

N = 100
X = np.random.uniform(-1.5, 1.5, N)

b0, b1, b2, b3, b4 = 1, 0.1, -0.02, 0.01, 0.5
y = b0 + b1*X + b2*(X**2) + b3*(X**2) + b4*(X**4) + np.random.normal(0, 0.4, N)

plt.scatter(X, y)
plt.show()

Como estamos com uma única feature, vamos aplicar o reshape já agora!

Podemos fazer uma regressão linear...

Naturalmente, temos métricas bem ruins, dada a escolha ruim de hipótese!

Hipótese atual:

$$f_{h, \  \vec{b}}(x) = b_0 + b_1x$$

Vamos fazer algo melhor: como nossos dados são aproximadamente quadráticos, faria sentido escolher uma **hipótese quadrática**, não é mesmo? Isto é,

$$f_{h, \  \vec{b}}(x) = b_0 + b_1x + b_2x^2$$

E é aqui que entra um dos aspectos mais importantes de um modelo linear como a regressão linear: **o modelo é linear nos parâmetros, não necessariamente nas features!**

Ou seja, o termo quadrático que incluímos **pode ser considerado como uma nova feature linear**. Para ver isso, basta definir $z \equiv x^2$, que voltamos a ter uma hipótese linear, mas agora em duas variáveis:

$$f_{h, \  \vec{b}}(x, z) = b_0 + b_1x + b_2z$$

Ou seja, ainda temos uma regressão linear (múltipla, agora).

E isso é verdade para **qualquer** combinação de features que possamos criar!

________

Um outro exemplo: considere uma hipótese linear para um modelo com duas features $x_1, x_2$:

$$f_{h, \  \vec{b}}(x_1, x_2) = b_0 + b_1x_1 + b_2x_2$$

Caso queiramos produzir um modelo quadrático, temos que incluir os termos $x_1^2, x_2^2$ e também $x_1x_2$ (que também é quadrático), de modo que nossa hipótese fica sendo:

$$f_{h, \  \vec{b}}(x_1, x_2) = b_0 + b_1x_1 + b_2x_2 + b_3 x_1^2 + b_4 x_2^2 + b_5 x_1 x_2$$

O que não deixa de ser uma **regressão linear múltipla** de 5 variáveis ($x_3 \equiv x_1^2$, $x_4 \equiv x_2^2$ e $x_5 \equiv x_1x_2$):

$$f_{h, \  \vec{b}}(x_1, x_2, x_3, x_4, x_5) = b_0 + b_1x_1 + b_2x_2 + b_3 x_3 + b_4 x_4 + b_5 x_5$$

E assim por diante! ;)

Assim, para criarmos um modelo quadrático para nossos dados, bastaria criarmos uma nova feature $z = x^2$, e passar apenas esta nova feature para o  modelo de regressão linear **simples**. Isso equivale a usar uma hipótese $$f_{h, \  \vec{b}}(z) = b_0 + b_1z = b_0 + b_1x^2$$

Vejamos:

No espaço transformado, esse foi o modelo treinado:

$f_{h, \  \vec{b}}(z) = b_0 + b_1z$

Mas, no espaço original, o modelo foi esse:

$f_{h, \  \vec{b}}(x) = b_0 + b_1x^2$

Agora sim, um modelo beeem melhor!!

E se quisermos usar a hipótese quadrática mais completa, com ambos os termos linear e quadrático? (Isto é, $f_{h, \  \vec{b}}(x) = b_0 + b_1x + b_2x^2$)

Bem simples: basta passarmos as duas features pro sklearn:

No espaço transformado, treinaríamos o seguinte modelo:

$f_{h, \  \vec{b}}(x, z) = b_0 + b_1x + b_2z$

Isso é um plano!

Mas, no espaço original, o modelo projetado seria esse:

$f_{h, \  \vec{b}}(x) = b_0 + b_2x + b_2x^2$

Uma hipótese quadrática genérica!

In [ ]:
# treinar o modelo em casa "manualmente", caso queira


No entanto, lembre que geramos os dados de acordo com um processo teório de grau 4! Então, seria legal que nossa hipótese tbm fosse de grau 4, nao é mesmo?

E isso é possível!

No geral, dá pra ir aumentando a ordem dos polinomios criando features de ordem maior uma a uma:

E aí, bastaria utilizar este df pra treinar o modelo!

In [ ]:
# treinar o modelo em casa "manualmente", caso queira

Mas esse é um procedimento bem manual. Pra nossa sorte, o sklearn existe, e uma de suas muitas ferramentas especiais para machine learning (no caso, pré-processamento) é o [polynomial features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), que permite a criação de toda as combinações polinomiais de features automaticamente!

O PolynomialFeatures é nosso primeiro exemplo de **transformer** do sklearn - um método cujo objetivo é aplicar alguma **transformação** aos dados. Veremos vários outros exemplos de transformers durante o curso.

Em particular, todos os transformers se comportam como se fossem "estimadores", no sentido de que eles devem 
ser "ajustados" aos dados -- por isso, eles também têm o método `.fit()` -- que ajusta o transformer aos dados; além do método `.transform()`, que efetivamente transforma os dados. Existe também o `.fit_transform()`, que faz as duas coisas ao mesmo tempo -- mas vamos evitar de usá-lo, por motivos que ficarão claros no futuro próximo (data leakage).

Lembre-se de fitar o transformados sempre nos dados de treino, apenas!

Vejamos o uso da classe:

Vamos generalizar nosss funções (veja no .py a definição da nova função `reg_lin_poly_features`)

Em aulas anteriores, discutimos sobre a **maldição da dimensionalidade**, e como é fácil overfitar um modelo ao aumentarmos a dimensionalidade (dado o correspondente aumento da complexidade da hipótese).

Vamos ver isso claramente, e fazer o plot do tradeoff viés-variância?

_____________
_____________
_____________

Agora que já entendemos a técnica em um dataset bem simples, vamos voltar pra um dataset real!

Vamos voltar pros dados da precificação de casas -- ali, o poly_features se mostrará ainda mais útil!

In [ ]:
df = pd.read_csv("../datasets/house_prices.csv")

X = df.drop(columns=["Id", "SalePrice"])
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_model = X_train.select_dtypes(include=np.number).dropna(axis="columns")
X_test_model = X_test.loc[:, X_train_model.columns]

Estes dois últimos modelos têm muuuuuito mais parametros que observações, portanto, aprenderam perfeitamente até mesmo os ruidos da base de treino!!

**Claro overfitting!**

Nossas hipóteses foram:

$$ f_{H, \vec{b}}(\vec{x}) = b_0 + b_1x_1 + b_2x_2 + \cdots + b_{594} x_{594}$$

pro primeiro modelo (features quadráticas); e, para o segundo (features cúbicas):

$$ f_{H, \vec{b}}(\vec{x}) = b_0 + b_1x_1 + b_2x_2 + \cdots + b_{7139} x_{7139}$$

Ou seja, temos um modelo **com muitos parâmetros**, ou seja, **muito complexo!**

Com tantos parâmetros assim, há muitos **graus de liberdade** pra que a hipótese se ajuste até às particularidades da base de treino... 

O resultado é evidente: temos um modelo altamente **overfitado**, dado o número enorme de features após o transformer -- e isso porque estamos utilizando apenas features quadráticas e cúbicas, imagine se tivéssemos usado features de grau maior!

É de se imaginar que muitas destas features não deveriam estar aí, não é mesmo?

Oras, uma forma interessante de eliminar features é fazendo o que chamamos de **feature selection**.

A ideia é a seguinte: gostaríamos sim de introduzir features polinomiais, aumentando um pouco a complexidade da hipótese, **mas não tanto!**. 

E é isso que conseguiremos fazer com as técnicas de **regularização**, que aprenderemos na próxima aula!

____
____
____